# <center> Studying Convergence

In [1]:
import bet.postProcess.compareP as cp
import bet.sampling.basicSampling as bsam
import bet.sample as sample
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wd
import bet.calculateP.calculateP as calcP
import scipy.spatial.distance as ds


In [2]:
def set_center(num_samples=100,
                   dim=2,
                   delta=0.1, reg=False):
    r"""
    Make unit square domain, create a hyperrectangle with sidelengths
    delta centered around `np.array([[0.5]]*dim)
    """
    s_set = sample.sample_set(dim)
    s_set.set_domain(np.array([[0, 1]]*dim))
    if reg:
        s = bsam.regular_sample_set(s_set, num_samples)
    else:
        s = bsam.random_sample_set('r', s_set, num_samples)
    dd = delta/2.0
    if dim > 1:
        probs = 1*(np.sum(np.logical_and(s._values <= (0.5+dd),
                                     s._values >= (0.5-dd)), axis=1) >= dim-1)
    else:
        probs = 1*(np.logical_and(s._values <= (0.5+dd),
                                     s._values >= (0.5-dd)))
    s.set_probabilities(probs/np.sum(probs))  # uniform probabilities
    s.estimate_volume_mc()
    s.global_to_local()
    return s

def set_zero(num_samples=100,
                   dim=2,
                   delta=0.1, reg=False):
    r"""
    Make unit square domain, create a hyperrectangle with sidelengths
    delta centered around `np.array([[0.5]]*dim)
    """
    s_set = sample.sample_set(dim)
    s_set.set_domain(np.array([[0, 1]]*dim))
    if reg:
        s = bsam.regular_sample_set(s_set, num_samples)
    else:
        s = bsam.random_sample_set('r', s_set, num_samples)    
    dd = delta
    if dim == 1:
        probs = 1*(s._values <= dd)
    else:
        probs = 1*(np.sum(s._values <= dd, axis=1) >= dim-1)
    s.set_probabilities(probs/np.sum(probs))  # uniform probabilities
    s.estimate_volume_mc()
    s.global_to_local()
    return s

def set_one(num_samples=100,
                   dim=2,
                   delta=0.1, reg=False):
    r"""
    Make unit square domain, create a hyperrectangle with sidelengths
    delta centered around `np.array([[0.5]]*dim)
    """
    s_set = sample.sample_set(dim)
    s_set.set_domain(np.array([[0, 1]]*dim))
    if reg:
        s = bsam.regular_sample_set(s_set, num_samples)
    else:
        s = bsam.random_sample_set('r', s_set, num_samples)
    
    dd = delta
    if dim == 1:
        probs = 1*(s._values >= (1-dd))
    else:
        probs = 1*(np.sum(s._values >= (1-dd), axis=1) >= dim-1)
    s.set_probabilities(probs/np.sum(probs))  # uniform probabilities
    s.estimate_volume_mc()
    s.global_to_local()
    return s

    
def answer_sq_hell(dim,d1=0.1,d2=0.2):
    f1 = d1**(-dim)
    f2 = d2**(-dim)
    s1, s2 = np.sqrt(f1), np.sqrt(f2)
    
    single = s2*(d2-d1)
    overlap = d1*(s1-s2)
    return (single + overlap)/2

def answer_tv(dim,d1=0.1,d2=0.2):
    f1 = d1**(-dim)
    f2 = d2**(-dim)
    
    single = f2*(d2-d1)
    overlap = d1*(f1-f2)
    return np.array(single + overlap)/2

# Testing Meshes and Sample Sizes

In [3]:
def preview1d(MC=100, left_N=10, right_N=10,
                  L_delta=0.1, R_delta=0.2,
                  left='r',right='r',
                  L_reg=False, R_reg=False, dim=1):
    if left =='r':
        L = set_one(left_N, dim, L_delta, L_reg)
    elif left=='l':
        L = set_zero(left_N, dim, L_delta, L_reg)
    else:
        L = set_center(left_N, dim, L_delta, L_reg)
        
    if right =='r':
        R = set_one(right_N, dim, R_delta, R_reg)
    elif right =='l':
        R = set_zero(right_N, dim, R_delta, R_reg)
    else:
        R = set_center(right_N, dim, R_delta, R_reg)
    
    np.random.seed(10)
    i = sample.sample_set(dim)
    i.set_domain(np.array([[0, 1]]*dim))
    I = bsam.random_sample_set('r', i, MC)
    
    L = cp.set_density(L)
    R = cp.set_density(R)
    mm = cp.metrization(I,L,R)
    mm.estimate_density()
    
#     print(check_densities(s,dim,delta1,0.05))
#     print(check_densities(r,dim,delta2,0.05))
    
    if dim == 1:
        plt.scatter(I._values, L._den)
        plt.scatter(I._values, R._den)
        plt.ylim(0,15)
        plt.hlines(5,0,1)
        plt.hlines(10,0,1)
    print(mm.distance(),mm.distance('sqhell'))
#     print(answer_tv(dim,delta1,delta2),
#           answer_sq_hell(dim,delta1,delta2))
#     print(0.5*np.sum(np.abs(mm._den_left - mm._den_right))/I.check_num())
wd.interact(preview1d, 
            MC=wd.IntSlider(500,100,10000,step=100, continuous_update=False),
            left_N=wd.IntSlider(500,100,10000,step=100, continuous_update=False),
            right_N=wd.IntSlider(500,100,10000,step=100, continuous_update=False),
            L_delta=wd.FloatSlider(0.1,min=0.1,max=.75,step=0.05),
            R_delta=wd.FloatSlider(0.1,min=0.1,max=.75,step=0.05),
            left=wd.Dropdown(options=['c','r','l']), right=wd.Dropdown(options=['c','r','l']),
            L_reg=wd.Checkbox(description="Regular Ref"),
            R_reg=wd.Checkbox(description="Regular Mesh"),
            dim=wd.IntSlider(1,1,5)
           )

interactive(children=(IntSlider(value=500, continuous_update=False, description='MC', max=10000, min=100, step…

<function __main__.preview1d(MC=100, left_N=10, right_N=10, L_delta=0.1, R_delta=0.2, left='r', right='r', L_reg=False, R_reg=False, dim=1)>

# Convergence Study

In [ ]:
n_l, n_r = 20, 1000
n = 1000

dim = 1
delta_l, delta_r = 0.4, 0.5 # order so d1 < d2
num_trials = 10

nvals = [10*2**N for N in range(2,9)]
hl = np.zeros((num_trials, len(nvals))) # data storage
tl = np.zeros((num_trials, len(nvals)))

i = sample.sample_set(dim)
i.set_domain(np.array([[0, 1]]*dim))
I = bsam.random_sample_set('r', i, n)

L = set_center(n_l, dim, delta1, reg=True)
R = set_center(n_r, dim, delta2, reg=False)
m = cp.metrization(I,L,R)
m.set_io_ptr_left()
m.set_io_ptr_right()

for t in range(num_trials):
    hh = [] # temporary vectors
    tv = []
    #     np.random.seed(t)
    for n in nvals:
        # effect of MC integration error
#         m.set_int(bsam.random_sample_set('r', i, n))
#         m.set_io_ptr_left()
#         m.set_io_ptr_right()
        
        # random sampling error relative to reference solution.
        r = set_center(n, dim, delta_r, reg=False)
        m.set_right(r)
        m.set_io_ptr_right()
        
        m.estimate_density()
        hh.append(m.distance('sqhell'))
        tv.append(m.distance('tv'))
        
    hl[t,:] = np.array(hh) # averaging, cumulative sum.
    tl[t,:] = np.array(tv)
h = np.mean(hl, axis=0)
t = np.mean(tl, axis=0)

In [ ]:
np.var(hl-2*answer_sq_hell(dim,delta1,delta2), axis=0)

In [ ]:
plt.figure(figsize=(20,10))
if delta1 == delta2:
    plt.plot(nvals, t, label='tv',lw=10,c='r')
    plt.plot(nvals, h, label='sq',lw=3,c='b',alpha=0.5)
else:
    plt.plot(nvals, np.var(hl-2*answer_sq_hell(dim,delta1,delta2), axis=0), ls = '--', label='err_h')
    plt.plot(nvals, np.var(tl-answer_tv(dim, delta1, delta2), axis=0), ls = '--', label='err_tv')
    plt.plot(nvals, np.mean(hl-2*answer_sq_hell(dim,delta1,delta2), axis=0), ls = '--', label='err_h')
    plt.plot(nvals, np.mean(tl-answer_tv(dim, delta1, delta2), axis=0), ls = '--', label='err_tv')
    
    # plt.plot(nvals, np.abs(hh-answer_tv(dim, delta1, delta2)))
    plt.plot(nvals, np.power(np.array(nvals,dtype=float),-1./2), lw=3, ls=':', label='${N}^{-1/(D+1)}$', c='k')
    
plt.yscale('log')
plt.xscale('log')
plt.legend(fontsize=20)
# plt.ylim(1E-4,1)
for k in range(4):
    plt.hlines(10**(-k),0,nvals[-1], lw=1,alpha=0.5)

In [ ]:
m.get_right(), r.check_num()

In [ ]:
h,answer_sq_hell(dim,delta1,delta2)

In [ ]:
t, answer_tv(dim, delta1, delta2)

In [ ]:
h[1:]/h[:-1]

In [ ]:
t[1:]/t[:-1]